https://github.com/davidrmiller/biosim4


In [1]:
from secrets import token_hex
import secrets
import random
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt

### funkcje neuronów

input

In [2]:
# Bliska przeszkoda
def close_obstacle(obj_loc_x, obj_loc_y, obj_list):
    select_obst = 0
    # iteruj po liście bez obiektu
    for obj_nr in obj_list:
        if obj_nr['coord'][0] - obj_loc_x == abs(1) and obj_nr['coord'][1] - obj_loc_y == abs(1):
            select_obst += 1
            
    return select_obst/8
       
# Daleka przeszkoda gdy ruch po osi x
def distant_obstacle_x_coord(obj_loc_x, obj_list):
    select_obst = 0
    # iteruj po liście bez obiektu
    for obj_nr in obj_list:
        if obj_nr['coord'][0] - obj_loc_x <= abs(5):
            select_obst += 1
            
    return select_obst/121

# Daleka przeszkoda gdy ruch po osi y
# to samo j.w.
def distant_obstacle_y_coord(obj_loc_y, obj_list):
    select_obst = 0
    # iteruj po liście bez obiektu
    for obj_nr in obj_list:
        if obj_nr['coord'][0] - obj_loc_y <= abs(5):
            select_obst += 1
            
    return select_obst/121


output

przykład jak zastosować neurony executeActions.cpp

In [3]:
def move_x(weight):
    weight_abs = abs(weight)
    x = np.random.choice(2, 1, p=[1-weight_abs, weight_abs])
    if weight < 0:
        x = x * -1
    return x

def move_y(weight):
    weight_abs = abs(weight)
    y = np.random.choice(2, 1, p=[1-weight_abs, weight_abs])
    if weight < 0:
        y = y * -1
    return y

def random_move():
    x,y = np.round(np.random.uniform(low=-1, high=1.0, size=2))
    return x,y
    

decode hexadecimal

In [4]:
def split_genome(hexval):
    binary = bin(int(hexval, 16))[2:]
    if len(binary) < 32:
        factor = 32 - len(binary)
        binary = '0' * factor + binary
  
    source_type, source_id = binary[0], binary[1:8]
    sink_type, sink_id = binary[8], binary[9:16]
    weight_sign, weight = binary[16], binary[17:]

    gene_value_list = {key:int(val, 2) for key, val in zip(['source_type', 'source_id', 'sink_type', 'sink_id', 'weight_sign', 'weight'],
                                          [source_type, source_id, sink_type, sink_id, weight_sign, weight])}
    
    return gene_value_list

def get_neurons_body(gen_component, nr_of_input, nr_of_inner, nr_of_actions):
    if gen_component['source_type'] == 0:
        input_id = gen_component['source_id'] % nr_of_input
        input_type = 'in'
    elif gen_component['source_type'] == 1:
        input_id = gen_component['source_id'] % nr_of_inner
        input_type = 'mid'

    if gen_component['sink_type'] == 0:
        output_id = gen_component['sink_id'] % nr_of_inner
        output_type = 'mid'
    elif gen_component['sink_type'] == 1:
        output_id = gen_component['sink_id'] % nr_of_actions
        output_type = 'out'

    if gen_component['weight_sign'] == 0:
        weight = (gen_component['weight'] /8191.25) * -1
    elif gen_component['weight_sign'] == 1:
        weight = gen_component['weight'] / 8191.25

    differ_neuron = f'{input_id}{input_type}{output_id}{output_type}'
    
    return input_id, input_type, weight, output_id, output_type, differ_neuron

def sum_weights_from_duplicated_neurons(gene_translated, l):
    if gene_translated:
        for neuron in gene_translated:
    #                     print('neuron', neuron.differ_neuron)
            if neuron.differ_neuron == l.differ_neuron:
                neuron.weight = neuron.weight + l.weight
            else:
                gene_translated.append(l)
                break
    else:
        gene_translated.append(l)

def generate_initial_genomes_for_population(nr_individuals, nr_of_genes, nr_of_input, nr_of_actions, nr_of_inner):
    lista = {}
    for nr_idividual in range(nr_individuals):
        gene_translated = []
        hexa_list = [token_hex(4) for i in range(nr_of_genes)]
        for l, hex_id in enumerate(hexa_list):

            gen_component = split_genome(hex_id)
            
            input_id, input_type, weight, output_id, output_type, differ_neuron = get_neurons_body(gen_component, nr_of_input, nr_of_inner, nr_of_actions)
            
            l = Neuron(hex_id, input_id, input_type, weight, output_id, output_type, differ_neuron)

            sum_weights_from_duplicated_neurons(gene_translated, l)

        lista[nr_idividual] = gene_translated
    return lista          

class Creature():
    def __init__(self, brain, x, y):
        self.brain = brain
        self.x = x
        self.y = y

class Neuron():
    def __init__(self, hex_id, input_id, input_type, weight, 
                 output_id, output_type, differ_neuron):
        self.hex_id = hex_id
        self.input_id = input_id
        self.input_type = input_type
        self.weight = weight
        self.output_id = output_id
        self.output_type = output_type
        self.differ_neuron = differ_neuron

hexadecimal generator

In [14]:
nr_of_input = 3
nr_of_actions = 3
nr_of_inner = 1
nr_of_genes = 4
nr_individuals = 4


res = generate_initial_genomes_for_population(nr_individuals, nr_of_genes, nr_of_input, nr_of_actions, nr_of_inner)

for l in res:
    for nr, i in enumerate(res[l]):
        print(nr, i.differ_neuron)
    print()


0 1in2out
1 0mid0mid
2 2in0mid
3 0in0mid

0 1in0out
1 0mid0mid
2 0mid0out
3 2in0out

0 2in0mid
1 0mid0mid
2 0mid0mid
3 0mid0out

0 0mid0mid
1 0in2out
2 0mid2out
3 0mid0mid



In [10]:
for nr, i in enumerate(res[1]):
    print(nr, i.differ_neuron)
#     print(i)


0 0mid0mid
1 0in1out
2 0mid0mid


### old data model

In [7]:
obj_list = {0:{'coord': [[2,3]], 'neuron':
               {'e20775ad': {'input': [0, 'mid'],'weight': -3.6777048679993896, 'output': [0, 'mid']},
                'ef9aa29c': {'input': [0, 'mid'], 'weight': 1.0816419960323516, 'output': [1, 'out']},
                '9bdf3966': {'input': [0, 'mid'], 'weight': -1.793865405157943,'output': [0, 'out']},
                'a079dd71': {'input': [0, 'mid'], 'weight': 2.920311307797955,'output': [0, 'mid']}}},
            1:{'coord': [[2,3]], 'neuron':
              {'b18c8e5e': {'input': [0, 'mid'],'weight': 0.4490157179917595,'output': [2, 'out']},
               'b8f967c7': {'input': [0, 'mid'],  'weight': -3.2433389287349303, 'output': [1, 'out']},
               '673cf9a3': {'input': [12, 'in'], 'weight': 3.8014954982450786, 'output': [0, 'mid']},
               'c1a2706c': {'input': [0, 'mid'], 'weight': -3.51350526476423, 'output': [4, 'out']}}}}
